#### NOTE: the result is based on the code base [abb9a21](https://github.com/UBC-MDS/test-creation/pull/136/commits/abb9a21828cb257ff8f2629261dc1ad64ad7dcb0), which is similar to the commit [69d61a9](https://github.com/UBC-MDS/test-creation/tree/69d61a9f5ac62baefca23ee293a6cd09fe41eeb2) in the main branch

In [ ]:
import scipy
import pickle
import yaml
import pandas as pd
import altair as alt
from collections import Counter

In [ ]:
def get_report(response):
    report = []
    for result in response.call_results:
        if result.parsed_response:
            resp = result.parsed_response['results']
            for item in resp:
                item['file'] = result.files_evaluated[0] 
                item['success'] = result.success
                report.append(item)
        else:
            report.append({
                'ID': '2.1', # FIXME
                'Title': '',
                'Requirement': '',
                'Observation': '',
                'Functions': [],
                'Evaluation': '',
                'Score': 0,
                'file': result.files_evaluated[0],
                'success': result.success
            })
    return pd.DataFrame(report)

def extract_file_and_scores(resp_path, verbose=False):
    if verbose:
        print(resp_path)
    with open(resp_path, 'rb') as file:
        response = pickle.load(file)
    report = get_report(response)
    df = (
        report
        .pivot(index='file', columns='ID', values='Score')
        .rename_axis(None, axis=1)
    )
    df['success'] = report.groupby(['file'])['success'].all()
    df['response_path'] = resp_path
    return df.reset_index()

def generate_stat_plot(df_repo__stat, ground_truth=None, facet_col='repo', repo=None, id=None):
    """
    Generate Stat plot across all repo and all checklist item
    Optional to incorporate ground truth and select specific repo/checklist item
    """
    if facet_col == 'repo':
        x_col = 'id'
        x_title = 'Checklist ID'
    elif facet_col == 'id':
        x_col = 'repo'
        x_title = 'Repository'
    
    # the base chart
    if repo:
        df_repo__stat = df_repo__stat.query(f'repo == "{repo}"')
    if id:
        df_repo__stat = df_repo__stat.query(f'id == "{id}"')
    
    base = alt.Chart().transform_calculate(
        min="max(0, datum.mean-datum.std)",
        max="min(1, datum.mean+datum.std)"
    )
    
    # generate the points
    points = base.mark_point(
        filled=True,
        size=50,
        color='black'
    ).encode(
        x=alt.X(f'{x_col}:O').axis(labelAngle=0).title(x_title),
        y=alt.Y('mean:Q').scale(domainMin=0, domainMax=1).title('Score'),
    )
    
    # generate the error bars
    errorbars = base.mark_errorbar().encode(
        x=f"{x_col}:O",
        y=alt.Y("min:Q").title('1 SD'),
        y2="max:Q"
    )

    plot = points + errorbars
    
    if ground_truth is not None:
        # generate points of ground truth
        if repo:
            ground_truth = ground_truth.query(f'repo == "{repo}"')
        if id:
            ground_truth = ground_truth.query(f'id == "{id}"')
        
        df_repo__stat = pd.merge(df_repo__stat, ground_truth, how='left', on=['repo', 'id'])
        
        gt_points = alt.Chart().mark_point(
            filled=True,
            size=100,
            color='green',
            shape="diamond"
        ).encode(
            x=alt.X(f'{x_col}:O'),
            y=alt.Y('score:Q')
        )

        plot += gt_points

    plot = alt.layer(
                plot,
                data=df_repo__stat
            ).properties(
                width=400,
            ).facet(
                column=f'{facet_col}',
                columns=2
            )

    return plot

### preprocess data

In [ ]:
checklist_ids = ['2.1', '3.2', '3.5', '4.2', '5.3', '6.1', '6.2']

result_path = '../draft/batch_run_results/record_combine.yml'
with open(result_path, 'r') as file:
    config = pd.DataFrame(yaml.safe_load(file))

# prepare score data by repo, run, file
tmp = [
    extract_file_and_scores(path) for path in config['response_path'] # FIXME: excluded deepchem
]
tmp = pd.concat(tmp, axis=0).reset_index(drop=True)

raw_df_repo_run_file = config.merge(tmp, on='response_path', how='left')

In [ ]:
# filter non-test files in qlib
df_repo_run_file = raw_df_repo_run_file.query('(repo != "qlib") | (file.str.contains("../data/raw/openja/qlib/tests/"))')

# prepare score data by repo, run
df_repo_run = df_repo_run_file.groupby(['repo', 'run']).agg({
    id: ['max'] for id in checklist_ids
})
df_repo_run.columns = [col[0] for col in df_repo_run.columns]
df_repo_run = df_repo_run.reset_index()

# prepare statistics of scores by repo
df_repo__stat = df_repo_run.groupby(['repo']).agg({
    id: ['mean', 'std', 'count'] for id in checklist_ids
})
df_repo__stat = pd.melt(df_repo__stat.reset_index(), id_vars=[('repo', '')])
df_repo__stat.columns = ['repo', 'id', 'stat', 'value']
df_repo__stat = (
    df_repo__stat.pivot(index=['repo', 'id'], columns='stat', values='value')
    .reset_index()
    .rename_axis(None, axis=1)
)

# prepare counting of scores by repo
df_repo__count = df_repo_run.groupby(['repo'])['2.1'].apply(Counter).reset_index()
for id in checklist_ids[1:]:
    df_repo__count = df_repo__count.merge(
        df_repo_run.groupby(['repo'])[id].apply(Counter).reset_index(),
        on=['repo', 'level_1'],
        how='outer'
    )

df_repo__count = df_repo__count.fillna(0)

### Runs Quality

#### 1. Some non-test files are included in the evaluation

For example, the `./nanodet/nanodet/trainer/task.py`

In [ ]:
raw_df_repo_run_file.query('repo == "nanodet"')['file'].unique()[:3]

#### 2. Evaluation on the file `magenta/magenta/models/music_vae/data_test.py` is always failed

In [ ]:
df_repo_run_file[~df_repo_run_file.success]['file'].unique()

#### 3. `DeepSpeech`, `lightfm` and `magenta` have the least (Python) test files

In [ ]:
df_repo_run_file.query('run == 1').groupby(['repo'])['file'].count().reset_index()

#### 4. The test files are not always in a `tests/` folder. Is it be good practice to always do that? Should it be one of the checklist item to ensure all tests placed under `tests/` folder?

For example, `magenta`

In [ ]:
df_repo_run_file.query('repo == "magenta"')['file'].unique()

### Findings on 8 repos

In [ ]:
df_repo_run_file.repo.unique()

#### 1. Overview of accuracy and consistency `lightfm` evaluation

Let the ground truth of the `lightfm` is as the [following](https://github.com/UBC-MDS/test-creation/blob/69d61a9f5ac62baefca23ee293a6cd09fe41eeb2/report/repo_human_evaluation/human_evaluation_report-lightfm.md):

In [ ]:
# Ground truth
ground_truth = pd.DataFrame([
    {'repo': 'lightfm', 'id': '2.1', 'score': 1},
    {'repo': 'lightfm', 'id': '3.2', 'score': 1},
    {'repo': 'lightfm', 'id': '3.5', 'score': 0},
    {'repo': 'lightfm', 'id': '4.2', 'score': 1},
    {'repo': 'lightfm', 'id': '5.3', 'score': 0.5},
    {'repo': 'lightfm', 'id': '6.1', 'score': 1},
    {'repo': 'lightfm', 'id': '6.2', 'score': 1},
    {'repo': 'qlib', 'id': '2.1', 'score': 0.5},
    {'repo': 'qlib', 'id': '3.2', 'score': 1},
    {'repo': 'qlib', 'id': '3.5', 'score': 0},
    {'repo': 'qlib', 'id': '4.2', 'score': 0.5},
    {'repo': 'qlib', 'id': '5.3', 'score': 1},
    {'repo': 'qlib', 'id': '6.1', 'score': 1},
    {'repo': 'qlib', 'id': '6.2', 'score': 1},
])
ground_truth[ground_truth.repo == 'lightfm']

In [ ]:
generate_stat_plot(df_repo__stat, ground_truth=ground_truth, repo="lightfm", facet_col='repo')

The distribution of the scores for each checklist items:

In [ ]:
df_repo__count.query('repo == "lightfm"')

**Observations**:
The system evaluation kind of aligns with our evaluation, that is,
 - for those items that we believe "Satisfied" (Score = 1), the system mostly output 0.5 or 1
 - for those items that we believe "Partially Satisfied" or "Not Satisfied", the system mostly output 0.5 or 0
 - some checklist items display high variance, e.g. 3.5, 5.3 and 6.1.

#### 2. Overview of `qlib`
Let the ground truth of the `qlib` is as the following (FIXME: to be confirmed):

In [ ]:
# Ground truth
ground_truth[ground_truth.repo == 'qlib']

In [ ]:
generate_stat_plot(df_repo__stat, ground_truth=ground_truth, repo="qlib", facet_col='repo')

In [ ]:
df_repo__count.query('repo == "qlib"')

**Observations**: 
- There are more disagreement between system and manual evaluation
  - especially for 5.3, 6.1, 6.2.
- The items consistency in this repo are not similar to those in `lightfm`.
  - e.g. Variance for 3.5 is greatly reduced. Variance for 3.2 becomes larger.
- However, `qlib` is not just a machine learning project, it also contains a software inside.
  - e.g. It has a lot of randomly generated data by itself, instead of reading a data to perform analysis, it seems to deviate from the objective of 2.1.

#### 3. The consistency for each checklist items
 - Why is it important?
   If the score of a particular item varies a lot when evaluating a repository, it might mean that its prompt (`Requirement`) is confusing to the LLM, or the checklist item itself is not well defined.

In [ ]:
df_repo__stat.pivot(index='id', columns='repo', values='std')

In [ ]:
alt.Chart(df_repo__stat).mark_boxplot().encode(
    x="std:Q",
    y='id:N'
).properties(
    height=200,
    width=400
)

**Observations**:
 - The evaluation of the checklist item 2.1 `Ensure Data File Loads as Expected` is usually stable.
   - When evaluating a repository, 50% of the time its standard deviation is smaller than 0.05, the smallest among the others.

Below shows the breakdown of item scores for each repository:  
(NOTE: only `lightfm` and `qlib` have ground truth, in green diamond)

In [ ]:
generate_stat_plot(df_repo__stat, ground_truth=ground_truth, facet_col='id')

**Observations**:
 - (TBC) The standard deviation for Item 3.5 and 5.3 shows great variation, which might imply that test cases in some repo might be confusing to LLM while some are clear.
 - (TBC) The standard deviation for Item 5.3, 6.1, 6.2 are relatively high and consistent, which might imply that there is room for refining the prompt to reduce consistency issue.

#### 4. The consistency for each checklist items, compared to the `lightfm`
 - Why is it important? We optimized the consistency of our system using `lightfm`. Therefore, we treat this repository as a benchmark. If a particular checklist item has a much worse consistency in other repository, that might mean that the prompt for that item is not generalizable.

Below shows the standard deviations in a 30 runs for each checklist item for each repository:

In [ ]:
stds = df_repo__stat[['repo', 'std', 'id']].pivot(index='repo', columns='id')
stds

In [ ]:
stds_p = stds.copy()
stds_p.columns = [col[1] for col in stds_p.columns]
stds_p = stds_p.reset_index()
stds_p = stds_p.melt(id_vars='repo', var_name='id')

In [ ]:
stds_p.head()

In [ ]:
# stripplot = (
#     alt.Chart(stds_p)
#     .mark_point(filled=True, size=100)
#     .transform_calculate( 
#         # Generate Gaussian jitter with a Box-Muller transform 
#         jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
#         # jitter='random()'
#     ).encode( 
#         y=alt.Y( 
#             'jitter:Q', 
#             title=None, 
#             axis=alt.Axis(ticks=False, grid=True, labels=False), 
#             scale=alt.Scale(), 
#         ), 
#         x=alt.X('value:Q'), 
#         color=alt.Color('repo:N'),
#         row=alt.Row( 
#             'id:N',
#             header=alt.Header(
#                 labelFontSize=16,
#                 labelAngle=0
#             )
#         ),
#         tooltip='repo'
#     ).configure_facet( 
#         spacing=0
#     ).configure_view( 
#         stroke=None
#     ).configure_axis( 
#         labelFontSize=16, 
#         titleFontSize=16
#     ).properties(
#         height=50, 
#         width=600
#     ) 
# )
    
# stripplot 

In [ ]:
def generate_jitterbox_plot(df_stds_p):
    """
    Generate jitterbox plot across all repo and all checklist item
    """
    box = alt.Chart().mark_boxplot(
        color='grey',
        opacity=0.5,
        size=20,
    ).encode(
        x=alt.X('value:Q').title('SD(Score)'),
        y=alt.Y('id:N', title=None, axis=alt.Axis(labelPadding=10, grid=False))
    )
    
    stripplot = alt.Chart().mark_circle(size=100).encode(
        y=alt.Y( 
            'id:N',
            axis=alt.Axis(ticks=False, grid=True, labels=True), 
            scale=alt.Scale(), 
        ), 
        x='value:Q',
        yOffset="jitter:Q",
        color=alt.Color('id:N', legend=None),
        tooltip='repo'
    ).transform_calculate(
        # Generate Gaussian jitter with a Box-Muller transform
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    )
    
    plot = alt.layer(
        box,
        stripplot,
        data=df_stds_p
    ).configure_view( 
        stroke=None
    ).configure_axis( 
        labelFontSize=16, 
        titleFontSize=16
    ).properties(
        height=300, 
        width=600
    ) 
    
    return plot

In [ ]:
generate_jitterbox_plot(stds_p)

In [ ]:
alt.Chart(df_repo__stat).mark_boxplot().encode(
    x="std:Q",
    y='id:N'
).properties(
    height=200,
    width=400
)

In [ ]:
# !pip install altair_catplot
# !pip install seaborn

In [ ]:
# import altair_catplot

# altair_catplot.catplot(
#     stds_p, 
#     transform ='jitterbox', 
#     mark ='point', 
#     encoding = dict(
#         x = alt.X('value:Q'), 
#         y = alt.Y('id:N'), 
#         color = alt.Color('repo:N')
#     ) 
# )

In [ ]:
F = stds.drop(index='lightfm') / stds.loc['lightfm']

base = alt.Chart(
    F.melt(ignore_index=False).reset_index()[['repo', 'id', 'value']]
).transform_calculate(
    benchmark="1",
    threshold=f"{scipy.stats.f.ppf(0.975, 29, 29)}"
)

point = base.mark_point(
    filled=True,
    size=100,
).encode(
    x=alt.X('value:Q').title("std ratio (c.f. lightfm)"),
    y='id:N',
    color='repo',
    tooltip='repo'
).properties(
    height=200,
    width=400
)

point \
+ base.mark_rule(color='black').encode(x="benchmark:Q") \
+ base.mark_rule(color='red').encode(x="threshold:Q")
# jitter instead of mark_point <-- prompt vs. repo problem?
# prompt: sd of checklist item for all repo is high
# repo: most of repo have low sd, the repo we're looking at has outlier

**Observations**:
 - The evaluation of the checklist item 3.2 `Data in the Expected Format` becomes much more unstable in most of other repositories.
 - That of the 2.1 is significantly unstable in the repo `paperless-ng`, `magenta` and `DeepSpeech`, but it may be due to the repo itself.

TODO: to look into the 3.2's scores.

#### TODO: Given ground truth == 1, distribution of system score?
#### TODO: Given ground truth == 0, distribution of system score?

In [ ]:
def generate_histogram_plot(df_repo_run_long, df_ground_truth=None, repo=None, id=None):
    """
    Generate histogram across all repo and all checklist item
    Optional to incorporate ground truth and select specific repo/checklist item
    """
    # data
    repo_data = df_repo_run_long.copy()
    if repo:
        repo_data = repo_data.query(f'repo == "{repo}"')
    if id:
        repo_data = repo_data.query(f'id == "{id}"')

    # base histogram chart
    base = alt.Chart().mark_bar().encode(
                x=alt.X('eval_score:Q', title='Score'), 
                y=alt.Y('count()'), 
                color=alt.value('grey'),
                size=alt.value(20),
            )
    
    if df_ground_truth is not None:
        # data
        gt_data = df_ground_truth.copy()
        if repo:
            gt_data = gt_data.query(f'repo == "{repo}"')
        if id:
            gt_data = gt_data.query(f'id == "{id}"')
        
        repo_data = pd.merge(repo_data, gt_data, how='left', on=['repo', 'id'])
        repo_data['is_equal_to_gt'] = repo_data['eval_score'] == repo_data['score']
        
        # base histogram chart
        base = base.encode(
                    color=alt.Color('is_equal_to_gt', scale=alt.Scale(range=['grey', 'green']), legend=None)
                )
        base += base.mark_text().encode(
            text=alt.value('Ground Truth'),
            x='score',
            size=alt.value(10),
            color=alt.value('green'),
        )

    plot = alt.layer(
                base,
                data=repo_data
            ).properties(
                width=200,
                height=200,
            ).facet(
                row='repo',
                column='id'
            )        
    
    return plot

#### Contingency Table

In [ ]:
df_repo_run_p = pd.melt(df_repo_run, id_vars=['repo', 'run'], var_name='id', value_name='eval_score')
df_repo_run_p = pd.merge(df_repo_run_p, ground_truth, how='inner', on=['repo', 'id'])
df_repo_run_p = df_repo_run_p.rename(columns={'score': 'ground_truth'})
pd.pivot_table(df_repo_run_p, values='run', index=['repo', 'ground_truth'], columns=['eval_score'], aggfunc='count', fill_value=0)

In [ ]:
# generate_histogram_plot(df_repo_run_p, df_ground_truth=ground_truth)